In [ ]:
import cv2
import numpy as np
import math
import time
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import pyautogui

In [ ]:
# Initialize hand detector and classifier
detector = HandDetector(maxHands=1)
classifier = Classifier('Model/keras_model.h5', 'Model/labels.txt')

# Set video parameters
offset = 20
imgSize = 300
labels = ["free_hand", "right", "left", "v_up", "v_down", "max", "min", "stop"]

# Open video file
video_path = "uploaded_video.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Video control variables
control_delay = 1  # Seconds between actions
last_action_time = time.time() - control_delay
paused = False

while cap.isOpened():
    if not paused:
        success, img = cap.read()
        if not success:
            print("End of video or error.")
            break
    else:
        success, _ = cap.read()  # Read the frame but don't advance

    imgOutput = img.copy()
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset: y + h + offset, x - offset: x + w + offset]

        if imgCrop.size == 0:
            print("Empty image crop. Skipping frame.")
            continue

        aspectRatio = h / w
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize

        prediction, index = classifier.getPrediction(imgWhite, draw=True)
        label = labels[index]

        # Perform actions based on detected label
        current_time = time.time()
        if label == "v_up" and current_time - last_action_time >= control_delay:
            print("Volume up")
            last_action_time = current_time
        elif label == "v_down" and current_time - last_action_time >= control_delay:
            print("Volume down")
            last_action_time = current_time
        elif label == "stop" and current_time - last_action_time >= control_delay:
            paused = not paused
            print("Pause/Play toggled")
            last_action_time = current_time
        elif label == "max" and current_time - last_action_time >= control_delay:
            print("Full screen")
            last_action_time = current_time
        elif label == "min" and current_time - last_action_time >= control_delay:
            print("Exit full screen")
            last_action_time = current_time
        elif label == "right" and current_time - last_action_time >= control_delay:
            current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame + 30)  # Skip 30 frames
            print("Skipped forward")
            last_action_time = current_time
        elif label == "left" and current_time - last_action_time >= control_delay:
            current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
            cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, current_frame - 30))  # Go back 30 frames
            print("Skipped backward")
            last_action_time = current_time

    # Display the video frame
    cv2.imshow("Video", imgOutput)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()